In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel(
    r"source_data/Тестовое_задание_Data_аналитик_ЦО_2025_.xlsx",
    sheet_name="Данные для задачи 3"
)

df.head()

,Бренд AX 1,Неделя,Выручка,Маржинальность
0,Shiseido,14,1.353686e+09,0.167824
1,Clarins,14,1.329934e+09,0.091348
2,Shiseido,15,1.267702e+09,0.169212
3,Clarins,15,9.747119e+08,0.145904
4,Clinique,15,4.982012e+08,0.140136


In [3]:
# Переименовываем для удобства
data = df.rename(columns={
    "Бренд AX 1": "brand",
    "Неделя": "week",
    "Выручка": "revenue",
    "Маржинальность": "margin_rate",
})

# Считаем валовую прибыль (маржа в деньгах) = выручка * маржинальность в %
data["gp"] = data["revenue"] * data["margin_rate"]

data.head()

,brand,week,revenue,margin_rate,gp
0,Shiseido,14,1.353686e+09,0.167824,2.271804e+08
1,Clarins,14,1.329934e+09,0.091348,1.214863e+08
2,Shiseido,15,1.267702e+09,0.169212,2.145103e+08
3,Clarins,15,9.747119e+08,0.145904,1.422140e+08
4,Clinique,15,4.982012e+08,0.140136,6.981598e+07


In [4]:
# Сводная таблица по неделям
totals = data.groupby("week").agg(
    revenue=("revenue", "sum"),
    gp=("gp", "sum"),
)
totals["margin_rate"] = totals["gp"] / totals["revenue"]

w_prev2, w_prev = 14, 15

rev_14, rev_15 = totals.loc[w_prev2, "revenue"], totals.loc[w_prev, "revenue"]
gp_14, gp_15 = totals.loc[w_prev2, "gp"], totals.loc[w_prev, "gp"]
m_14, m_15 = totals.loc[w_prev2, "margin_rate"], totals.loc[w_prev, "margin_rate"]

# Расчитываем дельту
delta_rev = rev_15 - rev_14
delta_gp = gp_15 - gp_14
delta_m_pp = (m_15 - m_14) * 100

summary = pd.DataFrame({
    "Неделя 14": [f"{rev_14:,.0f}", f"{gp_14:,.0f}", f"{m_14*100:.2f}%"],
    "Неделя 15": [f"{rev_15:,.0f}", f"{gp_15:,.0f}", f"{m_15*100:.2f}%"],
    "Изменение": [f"{delta_rev:,.0f}", f"{delta_gp:,.0f}", f"{delta_m_pp:+.2f} п.п."],
}, index=["Выручка", "Валовая прибыль", "Маржинальность"])

summary

,Неделя 14,Неделя 15,Изменение
Выручка,"8,464,188,705","8,207,907,632","-256,281,073"
Валовая прибыль,"1,237,987,713","1,268,186,314","30,198,601"
Маржинальность,14.63%,15.45%,+0.82 п.п.


In [5]:
# Создаём pivot по брендам
pivot = data.pivot_table(index="brand", columns="week", values=["revenue", "gp"], aggfunc="sum").fillna(0)

# Маржинальность по брендам
mr_14 = np.where(pivot[("revenue", w_prev2)] > 0, pivot[("gp", w_prev2)] / pivot[("revenue", w_prev2)], 0)
mr_15 = np.where(pivot[("revenue", w_prev)] > 0, pivot[("gp", w_prev)] / pivot[("revenue", w_prev)], 0)

# Конечная таблица по брендам
brands = pd.DataFrame({
    "rev_w14": pivot[("revenue", w_prev2)],
    "rev_w15": pivot[("revenue", w_prev)],
    "delta_rev": pivot[("revenue", w_prev)] - pivot[("revenue", w_prev2)],
    "margin_w14": mr_14,
    "margin_w15": mr_15,
    "delta_margin_pp": (mr_15 - mr_14) * 100,
    "gp_w14": pivot[("gp", w_prev2)],
    "gp_w15": pivot[("gp", w_prev)],
    "delta_gp": pivot[("gp", w_prev)] - pivot[("gp", w_prev2)],
}).sort_values("delta_gp", ascending=False)

# Топ 7 брендов по увеличению валовой прибыли
print("ТОП 7 БРЕНДОВ ПО РОСТУ ВАЛОВОЙ ПРИБЫЛИ (GP):")
print(brands[["delta_gp", "delta_rev", "margin_w14", "margin_w15"]].head(7).to_string())

print("\n\nТОП 7 БРЕНДОВ ПО ПАДЕНИЮ ВАЛОВОЙ ПРИБЫЛИ (GP):")
print(brands[["delta_gp", "delta_rev", "margin_w14", "margin_w15"]].tail(7).to_string())

ТОП 7 БРЕНДОВ ПО РОСТУ ВАЛОВОЙ ПРИБЫЛИ (GP):
                delta_gp     delta_rev  margin_w14  margin_w15
brand                                                         
Clarins     2.072776e+07 -3.552224e+08    0.091348    0.145904
Payot       1.966856e+07  1.022935e+08    0.216540    0.209797
Valmont     1.435105e+07  7.565795e+07    0.152891    0.163173
La Mer      8.121260e+06  4.601070e+07    0.172968    0.173562
St.Barth    6.431818e+06  3.458730e+07    0.193963    0.191233
La Prairie  6.045029e+06  1.018990e+07    0.154521    0.175854
Kiehls      3.870925e+06  2.095201e+07    0.193312    0.192444


ТОП 7 БРЕНДОВ ПО ПАДЕНИЮ ВАЛОВОЙ ПРИБЫЛИ (GP):
                       delta_gp     delta_rev  margin_w14  margin_w15
brand                                                                
Helena Rubinstein -3.374353e+06 -1.238216e+07    0.130000    0.080604
CHANEL            -3.979465e+06 -1.938396e+07    0.087772    0.077289
Dr Barbara Sturm  -3.984443e+06 -1.433499e+07    0.223325  

In [6]:
# Доля выручки каждого бренда в общей выручке недели
w14 = brands["rev_w14"] / rev_14
w15 = brands["rev_w15"] / rev_15

m14 = brands["margin_w14"]
m15 = brands["margin_w15"]

# Микс-эффект: вклад каждого бренда за счёт изменения доли выручки
mix_contrib_pp = ((w15 - w14) * m14) * 100

# Внутри-брендовый эффект: вклад за счёт изменения маржинальности бренда
within_contrib_pp = (w15 * (m15 - m14)) * 100

# Общие эффекты
mix_total_pp = mix_contrib_pp.sum()
within_total_pp = within_contrib_pp.sum()
delta_total_pp_check = (m_15 - m_14) * 100

print("Разложение изменения маржинальности:")
print(f"\nМикс-эффект (изменение доли выручки): {mix_total_pp:+.2f} п.п.")
print(f"Внутри-брендовый эффект (изменение маржи брендов): {within_total_pp:+.2f} п.п.")
print(f"Реальное изменение маржинальности: {delta_total_pp_check:+.2f} п.п.")

Разложение изменения маржинальности:

Микс-эффект (изменение доли выручки): +0.33 п.п.
Внутри-брендовый эффект (изменение маржи брендов): +0.49 п.п.
Реальное изменение маржинальности: +0.82 п.п.


In [7]:
# Таблица вкладов в маржинальность
drivers = pd.DataFrame({
    "mix_contrib_pp": mix_contrib_pp,
    "within_contrib_pp": within_contrib_pp,
    "total_contrib_pp": mix_contrib_pp + within_contrib_pp,
}).sort_values("total_contrib_pp", ascending=False)

print("Вклады брендов в изменение маржинальности:")
print(drivers.round(3).to_string())

# Определяем ключевые драйверы маожинальности
top_drivers_pos = drivers.head(3)
top_drivers_neg = drivers.tail(3)

print("\nТоп 3 улучшающих маржинальность:")
print(top_drivers_pos.round(3).to_string())

print("\nТоп 3 ухудшающих маржинальность:")
print(top_drivers_neg.round(3).to_string())

Вклады брендов в изменение маржинальности:
                   mix_contrib_pp  within_contrib_pp  total_contrib_pp
brand                                                                 
Clarins                    -0.351              0.648             0.297
Payot                       0.291             -0.030             0.261
Valmont                     0.152              0.034             0.186
La Mer                      0.112              0.002             0.113
La Prairie                  0.031              0.054             0.085
St.Barth                    0.087             -0.003             0.083
Kiehls                      0.063             -0.002             0.060
Caudalie                    0.115             -0.062             0.054
ELEMIS                      0.053             -0.002             0.051
OK Beauty                   0.046              0.003             0.048
Thalgo                      0.027              0.021             0.048
James Read                  0.033 

### Аналитическая записка

In [8]:
# А
top_gp_up = brands.head(3)
top_gp_down = brands.tail(3)
top_m_down = brands.sort_values("delta_margin_pp").head(3)

print("="*80)
print("АНАЛИТИЧЕСКАЯ ЗАПИСКА")
print("Анализ продаж: неделя 15 vs неделя 14")
print("="*80)

print(f"\nИтоговая метрика (TL;DR):")
print(f"\nВыручка изменилась на {delta_rev/rev_14*100:+.2f}% ({delta_rev:,.0f} руб.), валовая прибыль на {delta_gp/gp_14*100:+.2f}% ({delta_gp:+,.0f} руб.)")
print(f"Маржинальность изменилась с {m_14*100:.2f}% до {m_15*100:.2f}% ({delta_m_pp:+.2f} п.п.)")

print(f"\nКлючевые драйверы:")
print(f"Улучшение маржинальности на {delta_m_pp:.2f} п.п. обеспечено:")
print(f"  • Внутри-брендовым эффектом: {within_total_pp:+.2f} п.п. (сами бренды стали маржинальнее)")
print(f"  • Микс-эффектом: {mix_total_pp:+.2f} п.п. (сместилась структура продаж)")

print(f"\nТоп-3 бренда по росту валовой прибыли:")
for idx, (brand_name, row) in enumerate(top_gp_up.iterrows(), 1):
    print(f"  {idx}. {brand_name}: GP {row['delta_gp']:+,.0f} руб., маржа {row['margin_w14']:.1%}→{row['margin_w15']:.1%}")

print(f"\nТоп-3 бренда по падению валовой прибыли:")
for idx, (brand_name, row) in enumerate(top_gp_down.iterrows(), 1):
    print(f"  {idx}. {brand_name}: GP {row['delta_gp']:+,.0f} руб., маржа {row['margin_w14']:.1%}→{row['margin_w15']:.1%}")

print(f"\nПроблемные бренды (падение маржи):")
for idx, (brand_name, row) in enumerate(top_m_down.iterrows(), 1):
    print(f"  {idx}. {brand_name}: маржа {row['margin_w14']:.1%}→{row['margin_w15']:.1%} ({row['delta_margin_pp']:+.2f} п.п.)")

print(f"\nРекомендации (быстрый результат):")
print(f"1. Ограничить скидки/промо в брендах с падением маржи (особенно {top_m_down.index[0]})")
print(f"   Возможный возврат GP: {abs(top_m_down.iloc[0]['delta_margin_pp'])*rev_15/100:,.0f} руб. за неделю при восстановлении маржи")
print(f"2. Усилить продвижение маржинальных лидеров ({top_gp_up.index[0]}, {top_gp_up.index[1]})")
print(f"   Они дают максимальный вклад в прибыль")

print("\n" + "="*80)

АНАЛИТИЧЕСКАЯ ЗАПИСКА
Анализ продаж: неделя 15 vs неделя 14

Итоговая метрика (TL;DR):

Выручка изменилась на -3.03% (-256,281,073 руб.), валовая прибыль на +2.44% (+30,198,601 руб.)
Маржинальность изменилась с 14.63% до 15.45% (+0.82 п.п.)

Ключевые драйверы:
Улучшение маржинальности на 0.82 п.п. обеспечено:
  • Внутри-брендовым эффектом: +0.49 п.п. (сами бренды стали маржинальнее)
  • Микс-эффектом: +0.33 п.п. (сместилась структура продаж)

Топ-3 бренда по росту валовой прибыли:
  1. Clarins: GP +20,727,762 руб., маржа 9.1%→14.6%
  2. Payot: GP +19,668,557 руб., маржа 21.7%→21.0%
  3. Valmont: GP +14,351,054 руб., маржа 15.3%→16.3%

Топ-3 бренда по падению валовой прибыли:
  1. Sisley: GP -12,335,429 руб., маржа 15.5%→16.7%
  2. Clinique: GP -12,577,014 руб., маржа 17.0%→14.0%
  3. Shiseido: GP -12,670,085 руб., маржа 16.8%→16.9%

Проблемные бренды (падение маржи):
  1. Grown Alchemist: маржа 20.4%→0.0% (-20.37 п.п.)
  2. Dr Brandt: маржа 13.6%→0.0% (-13.64 п.п.)
  3. Bellefontaine: 